In [1]:
import requests
import pandas as pd

In [65]:
token = 'n0emNjRIkzyy4Z9Ii7HYYvCtani5AWT4T7HOcdrm'

paramaters = {'min_rank':'grand-champion','max_rank':'grand-champion','playlist':'ranked-doubles'}

head = {'Authorization': token}

metadata = ['id', 'link', 'rocket_league_id', 'replay_title', 'map_code',
            'map_name', 'playlist_id', 'playlist_name', 'duration', 'overtime',
            'overtime_seconds', 'season', 'season_type', 'date', 'date_has_tz',
            'visibility', 'created', 'min_rank.tier', 'min_rank.division',
            'min_rank.name', 'min_rank.id', 'max_rank.tier', 'max_rank.division',
            'max_rank.name', 'max_rank.id', 'uploader.steam_id', 'uploader.name',
            'uploader.profile_url', 'uploader.avatar']

In [191]:
def get_players_csv(id: str) -> pd.DataFrame:

    df = pd.read_csv(f'https://ballchasing.com/dl/stats/players/{id}/{id}-players.csv', sep=';')

    return df.T

In [182]:
def get_teams_csv(id: str) -> pd.DataFrame:

    df = pd.read_csv(f'https://ballchasing.com/dl/stats/teams/{id}/{id}-team-stats.csv', sep=';')

    return df.set_index('color').T

In [248]:
data = pd.DataFrame()

def new_data(criteria, auth, stuff) -> pd.DataFrame:

    api_request = requests.get('https://ballchasing.com/api/replays', params=criteria, headers=auth)
    infojson = api_request.json()

    df = pd.json_normalize(infojson, record_path=['list'], sep='.')
    # df = pd.concat([df, pd.json_normalize(infojson['list'], ['blue', 'players'], stuff, record_prefix='blue.player.', sep='.',errors='ignore')])
    # df = pd.concat([df, pd.json_normalize(infojson['list'], ['orange', 'players'], stuff, record_prefix='orange.player.', sep='.',errors='ignore')])

    df = df.drop(['blue.players','orange.players'], axis=1)

    for column_name in ['blue_team','blue_player1','blue_player2','orange_team','orange_player1','orange_player2']:
        df.insert(df.columns.size, column_name, pd.Series([]))
    
    
    list = []

    # for i,colNames in enumerate(['blue_player1','blue_player2','orange_player1','orange_player2']):
    #     for index,row in df.iterrows():
    #         list.append((get_players_csv(row['id']))[i])   sending too many http requests, need to add to this to the DataFrame periodically
    #     df.insert(df.columns.size, colNames, pd.Series(list))

    return df

**Data Cleaning** *OUTDATED*

* Data is received in a (trash) json format

* Everything is flattened at first except for the color.players column which still contains raw json text

* Each color of json text is flattened and then concatenated back into the main DataFrame

* Unneccesary columns are dropped

* Columns for individual players (4 total) and teams (2 total) are then made

* Data for individual players is downloaded and populated into a Series and placed into DataFrame cells

* Data for teams is downloaded and populated into a Series and placed into DataFrame cells

In [246]:
data = pd.concat([data, new_data(paramaters, head, metadata)]).reset_index(drop=True)
data.tail()

C:\Users\Charles\AppData\Local\Temp\ipykernel_26260\4176279611.py:15: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  df.insert(df.columns.size, column_name, pd.Series([]))
C:\Users\Charles\AppData\Local\Temp\ipykernel_26260\4176279611.py:15: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  df.insert(df.columns.size, column_name, pd.Series([]))
C:\Users\Charles\AppData\Local\Temp\ipykernel_26260\4176279611.py:15: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  df.insert(df.columns.size, column_name, pd.Series([]))
C:\Users\Charles\AppData\Local\Temp\ipykernel_26260\4176279611.py:15: FutureWarning: The default dtype for empty Series will be 'object'

,id,link,rocket_league_id,replay_title,map_code,map_name,playlist_id,playlist_name,duration,overtime,...,uploader.name,uploader.profile_url,uploader.avatar,orange.goals,blue.goals,overtime_seconds,blue_player1,blue_player2,orange_player1,orange_player2
45,eb9ec555-3c10-4e50-8dc1-112b71518b48,https://ballchasing.com/api/replays/eb9ec555-3...,025B920543914E7DA725C08F44F60D3B,2023-05-13.15.44 Rocksox Ranked Doubles Win,park_rainy_p,Beckwith Park (Stormy),ranked-doubles,Ranked Doubles,72,False,...,Rocksox,https://steamcommunity.com/id/Rocksox94/,https://avatars.akamai.steamstatic.com/a60fdfb...,NaN,3.0,NaN,NaN,NaN,NaN,NaN
46,887f80af-9756-4d03-bd5f-fed580f5c0ce,https://ballchasing.com/api/replays/887f80af-9...,0BCF50B34EBD583E3A9B3FBF8A2DB216,2023-05-13.17.44 amandinhaarl Ranked Doubles Loss,beach_night_p,Salty Shores (Night),ranked-doubles,Ranked Doubles,396,True,...,diaguiarg,https://steamcommunity.com/profiles/7656119843...,https://avatars.akamai.steamstatic.com/c15fa77...,4.0,3.0,19.0,NaN,NaN,NaN,NaN
47,90d0dcaf-9b53-436d-b786-acb313a572c5,https://ballchasing.com/api/replays/90d0dcaf-9...,B987EEC247016195D991A493A1E69C73,2023-05-13.13.44 King Pine Ranked Doubles Loss,outlaw_oasis_p,NaN,ranked-doubles,Ranked Doubles,242,False,...,King Pine,https://steamcommunity.com/id/KingPPine/,https://avatars.akamai.steamstatic.com/2674d57...,5.0,NaN,NaN,NaN,NaN,NaN,NaN
48,45c2ad55-8454-4ec4-a07e-c460833d2660,https://ballchasing.com/api/replays/45c2ad55-8...,1F6A0B9D48261D1FFA35669F940E749D,2023-05-13.13.44 Seaish Ranked Doubles Win,outlaw_oasis_p,NaN,ranked-doubles,Ranked Doubles,242,False,...,Seaish,https://steamcommunity.com/id/seaish/,https://avatars.akamai.steamstatic.com/ad025b5...,5.0,NaN,NaN,NaN,NaN,NaN,NaN
49,a7316ce2-1d37-4305-9d24-8c5bf0c7ce9c,https://ballchasing.com/api/replays/a7316ce2-1...,86056E914F04FF2999532EAFD9794BD0,2023-05-13.16.44 J3 Ranked Doubles Win,stadium_day_p,DFH Stadium (Day),ranked-doubles,Ranked Doubles,352,False,...,j3rome.twitch,https://steamcommunity.com/id/j3rome_twitch/,https://avatars.akamai.steamstatic.com/f267fbb...,4.0,2.0,NaN,NaN,NaN,NaN,NaN


In [228]:
data['id'].unique()
data[data['id'] == 'ef689260-e8c8-41cf-b498-74cad6773432']

,id,link,rocket_league_id,replay_title,map_code,map_name,playlist_id,playlist_name,duration,overtime,...,max_rank.division,max_rank.name,max_rank.id,uploader.steam_id,uploader.name,uploader.profile_url,uploader.avatar,orange.goals,blue.goals,overtime_seconds
45,ef689260-e8c8-41cf-b498-74cad6773432,https://ballchasing.com/api/replays/ef689260-e...,AE6F9D33418B06E71D7BC188ED3A4C97,2023-05-13.21.21 BrownDooDoo123 Ranked Doubles...,utopiastadium_p,Utopia Coliseum,ranked-doubles,Ranked Doubles,311,False,...,3.0,Champion III Division 3,champion-3,76561198988374229,Hamza4789,https://steamcommunity.com/profiles/7656119898...,https://avatars.akamai.steamstatic.com/9bba146...,5.0,2.0,NaN


In [247]:
data.columns

Index(['id', 'link', 'rocket_league_id', 'replay_title', 'map_code',
       'map_name', 'playlist_id', 'playlist_name', 'duration', 'overtime',
       'season', 'season_type', 'date', 'date_has_tz', 'visibility', 'created',
       'min_rank.tier', 'min_rank.division', 'min_rank.name', 'min_rank.id',
       'max_rank.tier', 'max_rank.division', 'max_rank.name', 'max_rank.id',
       'uploader.steam_id', 'uploader.name', 'uploader.profile_url',
       'uploader.avatar', 'orange.goals', 'blue.goals', 'overtime_seconds',
       'blue_player1', 'blue_player2', 'orange_player1', 'orange_player2'],
      dtype='object')

In [232]:
get_players_csv('40f64128-d24c-4fe3-a959-c6805fb9e29d').T.columns

Index(['color', 'team name', 'player name', 'platform', 'player id', 'car id',
       'car name', 'game duration', 'score', 'goals', 'assists', 'saves',
       'shots', 'shots conceded', 'goals conceded',
       'goals conceded while last defender', 'shooting percentage', 'bpm',
       'avg boost amount', 'amount collected', 'amount collected big pads',
       'amount collected small pads', 'count collected big pads',
       'count collected small pads', 'amount stolen', 'amount stolen big pads',
       'amount stolen small pads', 'count stolen big pads',
       'count stolen small pads', '0 boost time', '100 boost time',
       'amount used while supersonic', 'amount overfill total',
       'amount overfill stolen', 'avg speed', 'total distance',
       'time slow speed', 'percentage slow speed', 'time boost speed',
       'percentage boost speed', 'time supersonic speed',
       'percentage supersonic speed', 'time on ground', 'percentage on ground',
       'time low in air', 'percen

In [207]:
get_teams_csv('40f64128-d24c-4fe3-a959-c6805fb9e29d').T.columns

Index(['team name', 'game duration', 'score', 'goals', 'assists', 'saves',
       'shots', 'shots conceded', 'goals conceded', 'shooting percentage',
       'bpm', 'avg boost amount', 'amount collected',
       'amount collected big pads', 'amount collected small pads',
       'count collected big pads', 'count collected small pads',
       'amount stolen', 'amount stolen big pads', 'amount stolen small pads',
       'count stolen big pads', 'count stolen small pads', '0 boost time',
       '100 boost time', 'amount used while supersonic',
       'amount overfill total', 'amount overfill stolen', 'total distance',
       'time slow speed', 'time boost speed', 'time supersonic speed',
       'time on ground', 'time low in air', 'time high in air',
       'time powerslide', 'count powerslide', 'time behind ball',
       'time in front of ball', 'time defensive half', 'time offensive half',
       'time defensive third', 'time neutral third', 'time offensive third',
       'time ball poss